# Setup

- Accessing Dataset
- Clone Splice code

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
ln: failed to create symbolic link '/content/drive/MyDrive/Thesis/Thesis': File exists


In [ ]:
!git clone https://github.com/omerbt/Splice.git

In [ ]:
subtrain_image_dir = "/content/drive/MyDrive/Cat | CATFLW/splitted/subtrain"

# ST

In [ ]:
import os
import shutil
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output
from Splice.train_v2 import train_model2
import os
import shutil
from pathlib import Path
import imghdr
import logging
import datetime
from torchvision.transforms import ToPILImage, Resize
import tkinter as tk
from tkinter import filedialog


**overriden functions**

In [ ]:
#Note save_result & train_model where overriden with few changes

# save_result in (util.py) callback was changed to give a special name - based on chosen convention - for image to ease processing them later
# Last geenrated image (last epoch was given sufix _final to ease grepping last results)
def save_result2(image_t, file_name, dataroot, arg, cfg=None, epoch=0):
    extre="none"
    now = datetime.now()
    timestamp = now.strftime("%m_%d_%M")
    if cfg is not None: #anadil_changed
        opt = cfg['optimizer']
        lrate = cfg['lr']
        current_epoch = epoch
        epoch = cfg['n_epochs']
        extra = opt + "_" + str(lrate) + "_" + str(current_epoch) + "_" + str(epoch)


    image = ToPILImage()(image_t)
    path = Path(f"{dataroot}/out")
    merge_images_v2(arg[0], arg[1], image, f"{path}/{file_name}_{extra}_{timestamp}_v2.png");
    path = Path(f"{dataroot}/out")
    if epoch == current_epoch:
        print("Saving Last Image, Yahooo!");
        image.save(f"{path}/{file_name}_{timestamp}_final.png")
    else:
        image.save(f"{path}/{file_name}_{timestamp}.png")



def train_model2(dataroot, image_name, config, callback=None, arg=None): # Added arg to callback function
    cfg = config
    if dataroot is not None:
        cfg['dataroot'] = dataroot
    # set seed
    seed = cfg['seed']
    if seed == -1:
        seed = np.random.randint(2**32 - 1)  # Generate seed without dtype
        #print(f""Seed value: {seed}, Type: {type(seed)}"")  # Debugging line to confirm the type
        random.seed(seed)  # Set Python's random seed
        np.random.seed(seed)  # Set NumPy's random seed
        torch.manual_seed(seed)  # Set PyTorch's random seed
        print(f'running with seed: {seed}.')

    # create dataset, loader
    dataset = SingleImageDataset(cfg)

    # define model
    model = Model(cfg)

    # define loss function
    criterion = LossG(cfg)

    # define optimizer, scheduler
    optimizer = get_optimizer(cfg, model.netG.parameters())

    scheduler = get_scheduler(optimizer,
                              lr_policy=cfg['scheduler_policy'],
                              n_epochs=cfg['n_epochs'],
                              n_epochs_decay=cfg['scheduler_n_epochs_decay'],
                              lr_decay_iters=cfg['scheduler_lr_decay_iters'])

    with tqdm(range(1, cfg['n_epochs'] + 1)) as tepoch:
        for epoch in tepoch:
            inputs = dataset[0]
            for key in inputs:
                inputs[key] = inputs[key].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            losses = criterion(outputs, inputs)
            loss_G = losses['loss']
            log_data = losses
            log_data['epoch'] = epoch

            # update learning rate
            lr = optimizer.param_groups[0]['lr']
            log_data["lr"] = lr
            tepoch.set_description(f"Epoch {log_data['epoch']}")
            tepoch.set_postfix(loss=log_data["loss"].item(), lr=log_data["lr"])

            # log current generated entire image
            if epoch % cfg['log_images_freq'] == 0:
                img_A = dataset.get_A().to(device)
                with torch.no_grad():
                    output = model.netG(img_A)
                save_result_v2(output[0],image_name, cfg['dataroot'],arg, cfg, epoch) ##anadil_changed
                with open('./st.log', 'a') as f:
                    # Append the content
                    f.write("image_name: {} loss:{:.2f} \n".format(image_name, loss_G))
                print(f" current loss_G is {loss_G}")

                if callback is not None:
                    callback(output[0])

            loss_G.backward()
            optimizer.step()
            scheduler.step()


In [ ]:
#Change it to your working directory
working_dir = "/content/drive/MyDrive/Splice"
#The command os.chdir("/Splice") is used to change the current working directory to "/Splice".
os.chdir(working_dir)

DATAROOT = "./datasets/curr_pair"
path_dataroot = Path(DATAROOT)
# if os.path.exists(path_dataroot):
#   shutil.rmtree(path_dataroot)
dir_a = Path(f"{DATAROOT}/A")
dir_b = Path(f"{DATAROOT}/B")
# path_Outimage = Path(f"{DATAROOT}/out")



In [ ]:

output_randomization_file = "/content/drive/MyDrive/splitted/pairs.json"  # Replace with your output file path

In [ ]:

def run_style_transfer_with_pairs(content_dir, style_dir, pairs, start_index=0):

    # Load configuration
    with open("conf/default/config.yaml", "r") as f:
        cfg = yaml.safe_load(f)
    cfg['n_epochs'] = 4000  # Adjust the number of epochs if needed
    cfg['dataroot'] = './datasets/curr_pair/step5-train1'
    max_times = 100
    for i, pair in enumerate(pairs[start_index:], start=start_index):
        if max_times == i :
          return;
        content_index = pair['content_index']
        content_image = pair['content_image']
        style_index = pair['style_index']
        style_image = pair['style_image']

        selected_image = content_image
        random_choice = style_image

        print(f"\n \n Starting Style Transfer for Pair {i } - finish at {max_times+1}")
        print(f"Using content image: {selected_image}")
        print(f"Using style image: {random_choice}")

        # Clear previous images in 'A' and 'B' directories
        for f in os.listdir(dir_a):
            os.remove(os.path.join(dir_a, f))
        for f in os.listdir(dir_b):
            os.remove(os.path.join(dir_b, f))

        # Prepare paths
        content_image_path = os.path.join(content_dir, selected_image)
        style_image_path = os.path.join(style_dir, random_choice)
        dest_content_image = os.path.join(dir_a, selected_image)
        dest_style_image = os.path.join(dir_b, random_choice)

        # Copy the images to the directories
        shutil.copy(content_image_path, dest_content_image)
        shutil.copy(style_image_path, dest_style_image)

        def show_img_aux(image_path):
            image = tf.keras.preprocessing.image.load_img(image_path)
            image_array = tf.keras.preprocessing.image.img_to_array(image)
            plt.imshow(image_array.astype('uint8'))
            plt.axis('off')
            plt.show()
        # # Display images
        show_img_aux(dest_content_image)
        show_img_aux(dest_style_image)

        # Train model for the pair
        output_name = f"{selected_image}_{random_choice}___{content_index}_{style_index}_output"
        train_model2(
            DATAROOT,
            output_name,
            cfg,
            show_result_st,
            [dest_style_image, dest_content_image]
        )

    print("Style transfer completed for all specified pairs!")


start_index = 0  # Start processing from index 0 (change as needed)

run_style_transfer_with_pairs(subtrain_dirr, subtrain_dirr, pairs, 0)